<a href="https://colab.research.google.com/github/albatellouc3m/P1_AA/blob/main/P1_AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primeramente tenemos que leer los datos en local (archivos de google colab)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

datos_generales = pd.read_csv('./attrition_availabledata_10.csv.gz')
datos_test = pd.read_csv('./attrition_competition_10.csv.gz') # Datos para realizar el test

datos_generales.head()

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,BusinessTravel,Department,...,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,7.462297,13,3,3,3.0,3.0,3.0,48,Travel_Rarely,Research & Development,...,Y,13,8,1,18.0,2,8,7,7,No
1,7.488597,6,1,3,3.0,4.0,4.0,31,Travel_Frequently,Research & Development,...,Y,17,8,1,3.0,2,1,0,0,Yes
2,7.116541,16,2,4,1.0,1.0,3.0,36,Travel_Rarely,Research & Development,...,Y,24,8,0,6.0,6,5,0,3,No
3,9.709125,6,3,4,3.0,3.0,3.0,49,Travel_Rarely,Research & Development,...,Y,22,8,0,25.0,2,7,0,7,No
4,6.297729,6,4,3,3.0,2.0,3.0,40,Travel_Frequently,Research & Development,...,Y,13,8,1,8.0,3,8,3,7,No


# **EDA** **Simplificado**
Antes de realizar el preprocesado de datos, debemos analizar el set de datos para sacar información importante para el ejercicio. Los pasos de este análisis son los siguientes:

1.   ¿Cuantas variables e instancias hay?

  El set de availabledata contiene un total de 2940 instancias y 31 variables. Para calcularlo hemos utilizado el comando *shape*.






In [ ]:
from IPython.display import display

display(datos_generales.shape)

(2940, 31)

,0
hrs,float64
absences,int64
JobInvolvement,int64
PerformanceRating,int64
EnvironmentSatisfaction,float64
JobSatisfaction,float64
WorkLifeBalance,float64
Age,int64
BusinessTravel,object
Department,object


2.   Tipos de variables

 En el conjunto de datos propuesto para el ejercicio se pueden identificar varios tipos de variable. Para mostrarlos usamos el comando *dtype*, el cual te muestra cada variable con su tipo.



In [ ]:
from IPython.display import display

display(datos_generales.dtypes)

,0
hrs,float64
absences,int64
JobInvolvement,int64
PerformanceRating,int64
EnvironmentSatisfaction,float64
JobSatisfaction,float64
WorkLifeBalance,float64
Age,int64
BusinessTravel,object
Department,object


3. Variables categóricas con alta cardinalidad

  Debemos estudiar si hay variables categóricas con alta cardinalidad, es decir, variables que toman un gran número de valores distintos, ya que estas variables pueden causar problemas y hacer que el modelo sea más complejo.

  El código para identificarlas es el siguiente:

In [ ]:
for col in datos_generales.select_dtypes(include=['object']).columns:
    print(f'{col}: {datos_generales[col].nunique()} valores únicos')

BusinessTravel: 3 valores únicos
Department: 3 valores únicos
EducationField: 6 valores únicos
Gender: 2 valores únicos
JobRole: 9 valores únicos
MaritalStatus: 3 valores únicos
Over18: 1 valores únicos
Attrition: 2 valores únicos


4. ¿Que variables tienen valores faltantes y cuantos?

  Para saber que variables tienen valores faltantes, es decir que en alguna de las instancias no tenga valor asignado, utilizamos el comando *isnull*.

In [ ]:
datos_generales.isnull().sum() # Añadimos el sum para que nos muestre una cuenta total de valores nulos y no uno por uno

,0
hrs,0
absences,0
JobInvolvement,0
PerformanceRating,0
EnvironmentSatisfaction,15
JobSatisfaction,12
WorkLifeBalance,29
Age,0
BusinessTravel,0
Department,0


5. ¿Hay columnas constantes o de id?

  Columnas constantes son aquellas que tienen el mismo valor para todas las instancias del conjunto de datos.

  Por otro lado, columnas de id son aquellas que contienen identificadores únicos para cada instancia.

  En nuestro conjunto de datos, la primera columna es de ID y podemos usar el comando nunique. Ya que en la pregunta 3 lo necesitabamos, vimos en el resultado que la variable Over18 es constante, y por lo tanto, no aporta información útil en el desarrollo del ejercicio.
  

6.   Regresion o clasificación:

  Se trata de un problema de clasificación, ya que la columna de clases no contiene valores reales. Además vemos que está completamente desbalanceado ya que tenemos 474 valores de attrition 'No' y 2466 de 'Yes'. El código para esta resolución es el siguiente:

In [ ]:
# Creamos contadores para cada valor
contador_yes = 0
contador_no = 0

# Recorremos todas las instancias de Attrition y vemos cuantos de cada valor hay
for valor in datos_generales['Attrition']:
    if valor == 'Yes':
        contador_yes += 1
    elif valor == 'No':
        contador_no += 1

print(f'Yes: {contador_yes}')
print(f'No: {contador_no}')


Yes: 474
No: 2466


# DECIDIR CÓMO SE VA A REALIZAR LA EVALUACIÓN